In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings

In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)

In [14]:
embeddings = OllamaEmbeddings(model='nomic-embed-text')

vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)

In [15]:
retriever = vectordb.as_retriever()

In [18]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    'langsmith_search',
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [19]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

In [21]:
api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper)

In [22]:
tools = [wiki_tool, arxiv_tool, retriever_tool]

In [25]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='deepseek-r1:1.5b')

In [28]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

e:\conda\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [30]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [32]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [33]:
agent_executor.invoke({'input' : 'what is langsmith'})



> Entering new AgentExecutor chain...


ResponseError: registry.ollama.ai/library/deepseek-r1:1.5b does not support tools (status code: 400)